# Coiled Example

Download the seismic:

```
wget http://s3.amazonaws.com/open.source.geoscience/open_data/newzealand/Taranaiki_Basin/OPUNAKE-3D/OPUNAKE3D-PR3461-FS.3D.Final_Stack.sgy
```

In [1]:
import numpy as np
from dask.distributed import Client, TimeoutError
import xarray as xr
import dask.array as da
import zarr

In [2]:
# client.close()

In [3]:
client = Client()

In [4]:
from d2geo.attributes.CompleTrace import ComplexAttributes
from d2geo.attributes.Frequency import Frequency

complex_attr = ComplexAttributes()
freq_attr = Frequency()

# Open up the seismic

This is the part where you would open up the segy file into a dask array or if memory allows into a numpy array. The problem is that if we open up the file as a numpy array it blows up the memory. So we should be able to open up something by chunks through zarr or equivalent. You can open segy files with segio I think.

What I was doing is opening the 300 GB in parts and saving it into a zarr file. Then I was able to lazily open the zarr array.

In [5]:
segy_array = np.random.rand(500, 500, 500)
i, j, k = segy_array.shape

In [6]:
segy_array_da = da.from_array(segy_array)

In [7]:
segy_array_da

dask.array<array, shape=(500, 500, 500), dtype=float64, chunksize=(250, 250, 250), chunktype=numpy.ndarray>

# Trying with Mapblocks

At some point I tried doing by blocks, but I was also blowing up the memory. Let us know what method you think is best. You can kinda disregard the following two cells

In [8]:
# mapped = segy_array_da.map_blocks(func, chunked_vt_xar_ds_rechunked.seis)

In [9]:
# mapped.compute()

# Calculate an attribute from the SEGY 3D cube using plain compute on dask array

In [10]:
attr_result = complex_attr.sweetness(segy_array_da)

In [12]:
seismic_attribute_zarr = attr_result.to_zarr('./sweetness.zarr/', compute=False)

In [14]:
seismic_attribute_zarr.compute()

[None, None, None, None, None, None, None, None]

In this example the whole thing works because the randome numpy array is small. I have not been able to do this with something larger than memory and I am not sure where the problem is.

In [17]:
attr_result[0, :, :].compute()

array([[46.05247513, 13.70489119, 17.98015175, ...,  9.26917784,
         4.36170539,  7.66130168],
       [30.60276427,  9.3560402 , 21.39029285, ..., 10.4749639 ,
        39.2172672 , 26.185999  ],
       [33.67989767,  4.77453424, 14.1771855 , ..., 40.73924314,
        41.9418025 , 36.29210677],
       ...,
       [19.8632955 , 21.24406352,  9.81005301, ..., 36.47239702,
         7.75353751, 20.65567317],
       [22.66175286, 43.1407866 , 24.54853604, ..., 41.7313297 ,
        40.84649475, 38.60279664],
       [22.77914742,  8.91716972,  4.47121266, ..., 26.34539022,
        28.95402092, 30.82375783]])